# Analysis on ALPHA DIGITS  

In [ ]:
!pip install nbformat

In [ ]:
%run experiments.ipynb

## I- Effect of Layers and units

we create a special function : generate_symmetric_configurations to create liste of hidden layer we want to use for the experiment

In [ ]:
def generate_symmetric_configurations(min_layers, max_layers, min_neurons, max_neurons, step_neurons):
    """
    Generate symmetrical configurations for the DBN's hidden layers.

    Args:
    min_layers (int): Minimum number of hidden layers.
    max_layers (int): Maximum number of hidden layers.
    min_neurons (int): Minimum number of neurons per layer.
    max_neurons (int): Maximum number of neurons per layer.
    step_neurons (int): No increase in the number of neurons.

    Returns:
    List[List[int]]: List of symmetrical configurations of hidden layers.
    """
    configurations = []
    for num_layers in range(min_layers, max_layers + 1):
        for num_neurons in range(min_neurons, max_neurons + 1, step_neurons):
            half = num_layers // 2
            config = [num_neurons] * half + [num_neurons] * (num_layers - half)
            configurations.append(config)
    return configurations

## 1.RBM launch function

In [ ]:
def run_rbm_experiment(hidden_units_sizes, n_epochs=100, character_sets=[['A', 'B'], ['1', '2', '3', '4'], ['A', 'B', '1', '2']]):
    """
    Conducts an experiment with Restricted Boltzmann Machines (RBMs) on different character sets.

    Args:
        hidden_units_sizes (list): A list of sizes for the hidden layers to experiment with.
        n_epochs (int): The number of training epochs. Default is 100.
        character_sets (list of lists): A list containing different sets of characters to train RBMs on.

    This function trains an RBM for each combination of character set and hidden unit size,
    generates and saves images representing the learned features, and plots the results in a grid.
    """
    # Determine the unique number of units
    unique_units = sorted(hidden_units_sizes)

    # Prepare a grid of subplots
    fig, axes = plt.subplots(len(character_sets), len(unique_units), figsize=(len(unique_units) * 3, len(character_sets) * 3), squeeze=False)

    for row_idx, characters in enumerate(character_sets):
        data = read_alpha_digit(characters, file_path=ALPHA_DIGIT_PATH)

        for col_idx, num_units in enumerate(unique_units):
            print(f"\nTraining RBM with {num_units} hidden units on characters {characters}")
            rbm = RBM(n_visible=data.shape[1], n_hidden=num_units, random_state=42)
            rbm.train(data, learning_rate=0.1, n_epochs=n_epochs, batch_size=15, print_each=5000)

            # Generate and display an image
            generated_image = rbm.generate_image(n_samples=1)

            # Save the image
            save_path = f"../resultat/rbm/{num_units}_Units_{len(characters)}_Chars.npy"
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            np.save(save_path, generated_image)

            ax = axes[row_idx, col_idx]
            ax.imshow(generated_image[0].reshape(20, 16), cmap='plasma')
            ax.set_title(f"Units: {num_units}, N_Chars: {len(characters)}")
            ax.axis('off')

    plt.tight_layout()
    # Save the generated image
    directory_image = "../resultat/images/rbm"
    os.makedirs(directory_image, exist_ok=True)
    plt.savefig(f"{directory_image}/rbm_{len(characters)}_chars_Units_{num_units}_Layers_{character_sets}.png")
    plt.tight_layout()
    plt.show()


In [ ]:
hidden_units_sizes = [100, 200, 300, 400, 500, 600, 700]
run_rbm_experiment(hidden_units_sizes, n_epochs=1000, character_sets=['E'])
run_rbm_experiment(hidden_units_sizes, n_epochs=1000, character_sets=['A'])


## 2. DBM launch function

In [ ]:
import matplotlib.pyplot as plt

def run_dbm_experiment(hidden_layers_sizes, n_epochs=100, characters=['A', 'B', '1', '2']):
    """
    Conducts an experiment with Deep Belief Networks (DBNs) on a set of characters.

    Args:
        hidden_layers_sizes (list of lists): A list containing the sizes of hidden layers to experiment with.
        n_epochs (int): The number of training epochs. Default is 100.
        characters (list): The characters to use in the experiment. Default is ['A', 'B', '1', '2'].

    This function trains a DBN for each specified configuration of hidden layer sizes,
    generates and saves images representing the learned features, and plots the results in a grid.
    """
    # Load the data
    data = read_alpha_digit(characters, file_path=ALPHA_DIGIT_PATH)

    # Determine the maximum number of layers and the unique number of units
    max_layers = max(len(sizes) for sizes in hidden_layers_sizes)
    unique_units = sorted({sizes[0] for sizes in hidden_layers_sizes})

    # Prepare a grid of subplots
    fig, axes = plt.subplots(len(unique_units), max_layers, figsize=(max_layers * 3, len(unique_units) * 3), squeeze=False)

    # Initialize all axes as invisible; they will be activated when used
    for ax_row in axes:
        for ax in ax_row:
            ax.set_visible(False)

    for layer_sizes in hidden_layers_sizes:
        print(f"\nTraining DBN with hidden layers: {layer_sizes}")
        dbn = DBN(n_visible=data.shape[1], hidden_layer_sizes=layer_sizes, random_state=42)
        dbn.train(data, learning_rate=0.1, n_epochs=n_epochs, batch_size=16, print_each=1000000)

        # Generate and display an image
        generated_image = dbn.generate_image(n_samples=1)
        unit_idx = unique_units.index(layer_sizes[0])
        layer_idx = len(layer_sizes) - 2  # Index 0 for 2 layers, index 1 for 3 layers, etc.

        ax = axes[unit_idx][layer_idx]
        ax.set_visible(True)
        ax.imshow(generated_image[0].reshape(20, 16), cmap='plasma')
        ax.set_title(f"N_Layers: {len(layer_sizes)}, N_Units: {layer_sizes[0]}")
        ax.axis('off')

        # Save the generated image
        directory = f"../resultat/dbn/{layer_sizes[0]}_Units_{len(layer_sizes)}_Layers"
        os.makedirs(directory, exist_ok=True)
        np.save(f"{directory}/Units_{layer_sizes[0]}_Chars_{''.join(characters)}.npy", generated_image[0])

    # Save the figure
    plt.tight_layout()
    directory_image = "../resultat/images/dbn"
    os.makedirs(directory_image, exist_ok=True)
    plt.savefig(f"{directory_image}/dbn_{len(characters)}_chars_{layer_sizes[0]}_Units_{len(layer_sizes)}_Layers.png")
    plt.tight_layout()
    plt.show()


In [ ]:
# Exemple d'utilisation avec des configurations générées
configurations = generate_symmetric_configurations(min_layers = 2, max_layers = 5, min_neurons = 100, max_neurons = 700, step_neurons = 100)
run_dbm_experiment(configurations, n_epochs=1, characters=['Y'])

## I- Effect of the number of characters on reconstruction

### 1. RBM

we modify the above corresponding function to have plot adapted to our analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

def run_rbm_experiment(hidden_units_sizes, n_epochs=100, character_sets=[['A', 'B'], ['1', '2', '3', '4'], ['A', 'B', '1', '2']]):
    """
    Conducts an experiment with Restricted Boltzmann Machines (RBMs) on different character sets,
    generating multiple samples for each configuration.

    Args:
        hidden_units_sizes (list): A list of sizes for the hidden units to experiment with.
        n_epochs (int): The number of training epochs. Default is 100.
        character_sets (list of lists): A list containing different sets of characters to train RBMs on.

    This function trains an RBM for each combination of character set and hidden unit size,
    generates and displays five samples from each trained RBM, and saves the results.
    """
    unique_units = sorted(hidden_units_sizes)

    # Prepare a grid of subplots; each configuration now has 5 columns for the 5 samples
    fig, axes = plt.subplots(len(character_sets), len(unique_units) * 5, figsize=(len(unique_units) * 3 * 5, len(character_sets) * 3), squeeze=False)

    for row_idx, characters in enumerate(character_sets):
        data = read_alpha_digit(characters, file_path=ALPHA_DIGIT_PATH)

        for col_idx, num_units in enumerate(unique_units):
            print(f"\nTraining RBM with {num_units} hidden units on characters {characters}")
            rbm = RBM(n_visible=data.shape[1], n_hidden=num_units, random_state=42)
            rbm.train(data, learning_rate=0.1, n_epochs=n_epochs, batch_size=15, print_each=5000)

            # Generate 5 images
            generated_images = rbm.generate_image(n_samples=5)

            for sample_idx in range(5):
                ax = axes[row_idx, col_idx * 5 + sample_idx]
                ax.imshow(generated_images[sample_idx].reshape(20, 16), cmap='plasma')
                ax.set_title(f"N_chars {len(characters)}, Generation: {sample_idx + 1}")
                ax.axis('off')

                # Save each generated sample
                save_path = f"../resultat/rbm/{num_units}_Units_{len(characters)}_Chars_Sample_{sample_idx}.npy"
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                np.save(save_path, generated_images[sample_idx])

    plt.tight_layout()
    directory_image = "../resultat/images/rbm"
    os.makedirs(directory_image, exist_ok=True)
    plt.savefig(f"{directory_image}/rbm_samples.png")
    plt.show()


In [ ]:
# configuration = 2 layer with 200 units each
configurations_fixe = [200]
run_rbm_experiment(configurations_fixe, n_epochs=2, character_sets = [['E'],['E', 'O'], ['E', 'O', 'A'], ['E', 'O', 'A', '2'], ['E', 'O', 'A', '2', '7']])


### 2. DBM

we modify the above corresponding function to have plot adapted to our analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

def run_dbm_experiment(hidden_layers_sizes, n_epochs=100, character_sets=[['A', 'B'], ['1', '2', '3', '4'], ['A', 'B', '1', '2']]):
    """
    Conducts an experiment with Deep Boltzmann Machines (DBMs) on different character sets,
    generating multiple samples for each configuration.

    Args:
        hidden_layers_sizes (list of lists): A list containing the sizes of hidden layers to experiment with.
        n_epochs (int): The number of training epochs. Default is 100.
        character_sets (list of lists): A list containing different sets of characters to train DBMs on.

    For each character set, this function trains a DBM, generates and displays five samples,
    and saves the generated images and the complete figure for each set.
    """
    # Assumes there's only one configuration of hidden layers sizes provided
    layer_sizes = hidden_layers_sizes[0]

    # For each set of characters, generate and display images
    for characters in character_sets:
        data = read_alpha_digit(characters, file_path=ALPHA_DIGIT_PATH)

        # Initialize a new figure
        plt.figure(figsize=(15, 3))  # Adjusted size for the set of subplots

        print(f"\nTraining DBN with hidden layers: {layer_sizes}")
        dbn = DBN(n_visible=data.shape[1], hidden_layer_sizes=layer_sizes, random_state=42)
        dbn.train(data, learning_rate=0.1, n_epochs=n_epochs, batch_size=16, print_each=1000000)

        # Generate 5 images
        generated_images = dbn.generate_image(n_samples=5)

        for img_idx in range(5):
            ax = plt.subplot(1, 5, img_idx + 1)
            ax.imshow(generated_images[img_idx].reshape(20, 16), cmap='plasma')  # Ensure the shape is correct
            ax.set_title(f"N_chars {len(characters)}, generation: {img_idx + 1}")
            ax.axis('off')

        # Save the generated images
        directory = f"../resultat/dbn/{'_'.join([str(size) for size in layer_sizes])}_Units_{len(characters)}_Chars"
        os.makedirs(directory, exist_ok=True)
        for img_idx, img in enumerate(generated_images):
            np.save(f"{directory}/Sample_{img_idx}_Chars_{''.join(characters)}.npy", img)

        # Save the complete figure for this set of characters
        plt.tight_layout()
        directory_image = f"../resultat/images/dbn/{'_'.join(characters)}"
        os.makedirs(directory_image, exist_ok=True)
        plt.savefig(f"{directory_image}/dbn_{len(characters)}_chars_{'_'.join([str(size) for size in layer_sizes])}_Units.png")

    # Display all figures at the end of the loop
    plt.show()


In [ ]:
# Example fixed configuration: two layers with 200 units each
fixed_configuration = [[400, 400, 400, 400]]

# Run the experiment with the fixed configuration and different character sets
run_dbm_experiment(fixed_configuration, n_epochs=2, character_sets=[['E'],['E', 'Y'], ['E', 'Y', 'A'], ['E', 'Y', 'A', '2'], ['E', 'Y', 'A', '2', '7']])


# read saved files for RBM and DBM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def load_and_display_image(file_path):
    """
    Charge et affiche une image à partir d'un fichier .npy.
    
    Args:
    file_path (str): Le chemin du fichier .npy à charger.
    """
    # Charger l'image à partir du fichier .npy
    image = np.load(file_path)
    
    image_data_reshaped = image.reshape((20, 16))

    # Afficher l'image
    plt.imshow(image_data_reshaped, cmap='plasma') # ou 'gray ou 'viridis' ou 'inferno' ou 'plasma' ou 'magma' ou 'cividis')
    plt.title("Loaded Image")
    plt.axis('off')  # Désactiver les axes pour une meilleure visualisation
    plt.show()

# Exemple d'utilisation
file_path = "../resultat/dbn/100_Units_2_Layers/Units_100_Chars_A.npy"
load_and_display_image(file_path)

file_path2 = "../resultat/rbn/100_Units_2_Chars.npy"
load_and_display_image(file_path2)